In [1]:
from re import X
import cv2
import mediapipe as mp
import numpy as np
import math

# 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0) # 기본캠 설정
cap.set (3, 1280) # 윈도우 창 가로길이
cap.set (4, 720) # 윈도우 창 세로길이
colorR = (125, 255, 50) # 사각형 기본컬러 설정
colorC = (255,0,0)

cx, cy, r_w, r_h = 100,100,200,200 # 동적 변수로 들어가기 위해 사각형 치수를 따로 변수에 담음

# 클래스 지정
class DragRect():
  # 메소드 생성 ( 초기화를 위한 메소드 )
  def __init__(self, posCenter, size = [200,200]): # 사각형의 좌표를 지정 ( posCenter = cx, xy / size = r_w, r_h )
    self.posCenter = posCenter 
    self.size = size

  # 메소드 생성 ( 직접 내부에서 쓰기 위한 메소드 )
  def update(self, cursor):
      cx, cy = self.posCenter
      r_w, r_h = self.size

      # 만약 검지손가락 끝이 사각형 영역에 있다면 self.posCenter 를 cursor의 좌표로 변경 ?
      # ( 사각형 내부에 내 검지손가락 좌표가 있는지 확인하는걸 가져다 쓰기위해 아래에서 썼던 if문을 가져옴 )
      if cx-r_w//2 < cursor[0] < cx+r_w//2 and cy-r_h//2 < cursor[1] < cy+r_h//2:
        self.posCenter = cursor

rectList = []
for x in range(5):
  rectList.append(DragRect([x*250+150,150]))

# 코너 사각형 함수 추가
def cornerRect(img, bbox, l=30, t=5, rt=1,
               colorR=(255, 0, 255), colorC=(0, 0, 255)):
    """
    :param img: Image to draw on.
    :param bbox: Bounding box [x, y, w, h]
    :param l: length of the corner line
    :param t: thickness of the corner line
    :param rt: thickness of the rectangle
    :param colorR: Color of the Rectangle
    :param colorC: Color of the Corners
    :return:
    """
    x, y, w, h = bbox
    x1, y1 = x + w, y + h
    if rt != 0:
        cv2.rectangle(img, bbox, colorR, rt)
    # Top Left  x,y
    cv2.line(img, (x, y), (x + l, y), colorC, t)
    cv2.line(img, (x, y), (x, y + l), colorC, t)
    # Top Right  x1,y
    cv2.line(img, (x1, y), (x1 - l, y), colorC, t)
    cv2.line(img, (x1, y), (x1, y + l), colorC, t)
    # Bottom Left  x,y1
    cv2.line(img, (x, y1), (x + l, y1), colorC, t)
    cv2.line(img, (x, y1), (x, y1 - l), colorC, t)
    # Bottom Right  x1,y1
    cv2.line(img, (x1, y1), (x1 - l, y1), colorC, t)
    cv2.line(img, (x1, y1), (x1, y1 - l), colorC, t)

    return img

with mp_hands.Hands(
  model_complexity = 0,
  max_num_hands = 1, # 손 하나만 잡아주는 옵션 추가 디폴트가 ( 2 )
  min_detection_confidence = 0.5,
  min_tracking_confidence = 0.5,) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.flip(image, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        # 기본적으로 미디어파이프 사이트에 있는 손 좌표 잡는 코드
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        # 검지 끝 엄지 끝 좌표잡기
        index_finger_tip_x = hand_landmarks.landmark[8].x
        index_finger_tip_y = hand_landmarks.landmark[8].y
        middle_finger_tip_x = hand_landmarks.landmark[12].x
        middle_finger_tip_y = hand_landmarks.landmark[12].y

        # 이미지 전체 크기를 바탕으로 실제 좌표로 반환함 ( 픽셀값 )
        h, w, _ = image.shape
        index_finger_tip = (int(index_finger_tip_x*w), int(index_finger_tip_y*h))
        middle_finger_tip = (int(middle_finger_tip_x*w), int(middle_finger_tip_y*h))

        # 손가락 사이 거리 구하기 ( 픽셀값 )
        distance = math.dist((index_finger_tip[0],index_finger_tip[1]), (middle_finger_tip[0], middle_finger_tip[1]))
        print(distance) # 검지 엄지 사이 거리 표시
        cv2.putText(image, f'{int(distance)} px', # 손가락 사이의 거리를 화면상에 텍스트로 표시 ( int로 표시해 소수점을 없앰 ) 
        (middle_finger_tip[0]-30,middle_finger_tip[1]-75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        # 40픽셀 이하로 거리좁히면 클릭 프린트
        if distance < 40:
          cursor = index_finger_tip # 검지 손가락 끝 랜드마크
          cv2.putText(image, "click !" , (index_finger_tip[0], index_finger_tip[1]-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
          # 업데이트 메서드 불러오기 ( 업데이트 메서드에 포함시킨 if문 )
          for rect in rectList:
            rect.update(cursor)

        
        # 좌표로 잡은 검지 끝, 엄지 끝 잡아서 원 그리기
        cv2.circle(image, index_finger_tip, 20, (0,0,0), -1, cv2.LINE_AA)
        cv2.circle(image, middle_finger_tip, 20, (0,0,0), -1, cv2.LINE_AA)
        # cv2.circle(image, , 20, (0,0,0), 2, cv2.LINE_AA)
        # 손가락 사이 거리에 선 긋기
        cv2.line(image, index_finger_tip, middle_finger_tip, (0,0,0), 2, 8)
    
    ### 기존의 사각형 그리기 ###
    # for rect in rectList:
    #   # 지정한 클래스로 사각형 그리기
    #   cx, cy = rect.posCenter
    #   r_w, r_h = rect.size
    #   # 드래그 앤 드롭할때 쓸 사각형 그리기
    #   cv2.rectangle(image, (cx-r_w//2,cy-r_h//2), (cx+r_w//2,cy+r_h//2), colorR, cv2.FILLED)

    ### 투명도 사각형 그리기 ###
    imgNew = np.zeros_like(image, np.uint8)
    for rect in rectList:
      # 지정한 클래스로 사각형 그리기
      cx, cy = rect.posCenter
      r_w, r_h = rect.size
      # 드래그 앤 드롭할때 쓸 사각형 그리기
      cv2.rectangle(imgNew, (cx-r_w//2,cy-r_h//2), (cx+r_w//2,cy+r_h//2), colorR, cv2.FILLED)
      cornerRect (imgNew, (cx-r_w//2, cy-r_h//2, r_w, r_h), 20, rt = 0) # 코너에 라인추가 하는 함수 불러오기

    out = image.copy()
    alpha = 0.25
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(image, alpha, imgNew, 1 - alpha, 0) [mask]
    
    # Flip the image horizontally for a selfie-view display.
    # cv2.imshow('MediaPipe Hands', cv2.flip(image, 1)) # << 원래 이미지 ( 과정 알아보기 쉽게 주석처리함 )
    cv2.imshow('MediaPipe Hands', out) # 투명도 박스로 교체하기 위해 원래 이미지에서 out을 출력하는걸로 바꿈
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()

23.08679276123039
20.518284528683193
22.847319317591726
16.1245154965971
10.44030650891055
6.4031242374328485
6.708203932499369
5.385164807134504
6.324555320336759
6.324555320336759
5.0990195135927845
3.1622776601683795
12.206555615733702
6.324555320336759
4.47213595499958
3.605551275463989
7.280109889280518
5.385164807134504
5.830951894845301
8.06225774829855
8.54400374531753
6.324555320336759
9.055385138137417
9.055385138137417
10.04987562112089
6.324555320336759
6.708203932499369
5.385164807134504
8.0
3.0
7.0
8.06225774829855
6.324555320336759
9.219544457292887
15.620499351813308
15.620499351813308
3.1622776601683795
1.4142135623730951
4.47213595499958
2.23606797749979
7.810249675906654
6.708203932499369
5.830951894845301
7.0710678118654755
11.40175425099138
14.317821063276353
12.083045973594572
8.06225774829855
5.0
10.295630140987
6.0
7.280109889280518
7.810249675906654
9.219544457292887
23.53720459187964
23.0
14.866068747318506
8.48528137423857
24.166091947189145
22.84731931759172